In [13]:
from tqdm import tqdm
tqdm.pandas()

import pandas as pd
import numpy as np

In [2]:
import cohere
import uuid
from typing import List, Dict

In [11]:
co = cohere.Client(api_key="e7FTxIVWjp0NEHTvIg34DYK89kE62P5mITf0FNqw")

In [19]:
# from unstructured.partition.html import partition_html
# from unstructured.chunking.title import chunk_by_title

# def load_and_chunk(row):
#     """
#     Loads the text from the sources and chunks the HTML content.
#     """
#     try:
#         elements = partition_html(url=row["link"])
#         chunks = chunk_by_title(elements, max_characters=2000)
#         return [str(chunk) for chunk in chunks]
#     except:
#         print(row["link"])
#         return []

In [ ]:
df = pd.read_json("/Users/shubham.chandel/Documents/github/links/client/public/links.jsonl", lines=True)

In [14]:
# df['prompt'] = df.apply(lambda x: f"Title: {x.subject}\nLink: {x.link}\n" + "\n".join(x.docs[:20]), axis=1)
df['prompt'] = df.apply(lambda x: f"Title: {x.subject}\nLink: {x.link}", axis=1)
df['embeddings'] = co.embed(texts=list(df.prompt.values), model="embed-english-v3.0", input_type="search_document").embeddings


In [24]:
df.to_pickle("df.embedding")

In [29]:
df = pd.read_pickle("df.embedding")
df.shape

(858, 5)

In [33]:
import requests

term = "jax learn"
url = "https://sksq96--search-app-searchapp-search-dev.modal.run"
params = {"term": term}

response = requests.get(url, params=params)
response.raise_for_status()  # Raise an exception for bad status codes
len(response.json())

858